# Examine how well we have mapped Hetionet to the UMLS

In [1]:
import pandas as pd

## Read Hetionet nodes

In [2]:
hnodes = pd.read_csv("hetionet/hetio_nodes.tsv", sep='\t')

In [3]:
hnodes.shape

(58700, 4)

In [4]:
hnodes.head()

,hetio_id,name,het_type,cui
0,DB00795,Sulfasalazine,Compound,UMLS:C0036078
1,DB00795,Sulfasalazine,Compound,UMLS:C0699547
2,DB00795,Sulfasalazine,Compound,UMLS:C4255898
3,N0000000151,Histamine H2 Receptor Antagonists,Pharmacologic Class,UMLS:C2757005
4,100996420,DNM1P50,Gene,100996420


## Some mapping statistics

In [5]:
temp = (hnodes
    .assign(iscui = lambda df: df["cui"].str.startswith("UMLS"))
    [["hetio_id", "het_type", "iscui"]]
    .drop_duplicates()
)

In [6]:
temp.groupby("het_type")["iscui"].value_counts().to_frame()

iscui
het_type            iscui       
Anatomy             True     402
Biological Process  True   11321
                    False     60
Cellular Component  True    1385
                    False      6
Compound            True    1544
                    False      8
Disease             True     135
                    False      2
Gene                True   19383
                    False   1562
Molecular Function  True    2871
                    False     13
Pathway             False   1822
Pharmacologic Class True     344
                    False      1
Side Effect         True    5734
Symptom             True     438

In [7]:
(temp
    .groupby("het_type")
    ["iscui"]
    .value_counts(normalize=True)
    .multiply(100)
    .to_frame()
)

iscui
het_type            iscui            
Anatomy             True   100.000000
Biological Process  True    99.472806
                    False    0.527194
Cellular Component  True    99.568656
                    False    0.431344
Compound            True    99.484536
                    False    0.515464
Disease             True    98.540146
                    False    1.459854
Gene                True    92.542373
                    False    7.457627
Molecular Function  True    99.549237
                    False    0.450763
Pathway             False  100.000000
Pharmacologic Class True    99.710145
                    False    0.289855
Side Effect         True   100.000000
Symptom             True   100.000000

The vast majority of concepts were mapped from Hetionet to the UMLS. The main category which was ignored was the pathways class. One contributing factor to unmapped concepts was the fact that some concepts have degraded and been removed from their source ontologies since creation of Hetionet.

These concepts will need to be upgraded to their latest versions before they can be harmonized with the UMLS.